<h1>1. Model Evaluation</h1>

<h2>1. Classification</h2>
- Confusion matrix<br>
- Precision<br>
- recall<br>
- ROC<br>
- f1-score<br>

<h2>2.Regression</h2>
- Mean Squared Error<br>
- Root Mean Squared Error<br>
- Mean Absolute Error<br>
- R^{2}<br>

<h1>Code</h1>

In [ ]:
import os
from glob import glob
from tqdm import tqdm
import pickle
import gc
gc.enable()

import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import matplotlib.pyplot as plt
import kaleido

import tensorflow as tf
import tensorflow.experimental.numpy as tfnp
# from lstnet_tensorflow.model1 import LSTNet, build_fn
from lstnet_tensorflow.tcn_model1 import custom_TCN, build_fn

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0],'GPU')
tf.config.experimental.set_memory_growth(gpus[0],True)

In [ ]:
def l1(y_true, y_pred):
    return tfnp.sum(abs(y_true - y_pred))


def l2(y_true, y_pred):
    return tfnp.sum(tfnp.square(y_true - y_pred))



# custom metric function의 Parameter는 y_true, y_pred로 고정되어야 함

# root_relative_squared_error (rrse)
def rrse(y_true, y_pred):
    return tfnp.sqrt(tfnp.sum(tfnp.square(y_true - y_pred)) / tfnp.sum(tfnp.square(y_true - tfnp.mean(y_true))))

# relative_absolute_error (rae)
def rae(y_true, y_pred):
    return tfnp.sum(tfnp.abs(y_true - y_pred)) / tfnp.sum(tfnp.abs(y_true - tfnp.mean(y_true)))

# empirical correlation coefficient (corr)
def corr(y_true, y_pred):
    vx = y_pred - tfnp.mean(y_pred)
    vy = y_true - tfnp.mean(y_true)
    cov = tfnp.sum(vx * vy)
    return cov / (tfnp.sqrt(tfnp.sum(vx ** 2)) * tfnp.sqrt(tfnp.sum(vy ** 2)))

In [ ]:
forecast_period = 100
lookback_period = 300
learning_rate = 0.001

metric = [rrse, rae, corr, 'mse']
metric_name = 'rrse_rae_corr_mse'
adversarial_checker = False # adversarial으로 fit하면 True, 아니면 False
epoch = 100
scaler = 'standard'  # normalize, standard, minmax

# data_names = ['adversarial_electricity', 'adversarial_solar_AL', 'adversarial_exchange_rate', 'adversarial_traffic']
data_names = ['adversarial_electricity', 'adversarial_solar_AL', 'adversarial_exchange_rate']
losses = [l1, l2]
loss_names = ['l1', 'l2']
epsilons = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

model_name = 'TCN'

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="loss",
        min_delta=0.0001,
        patience=10,
        verbose=1,
        mode="auto",
        baseline=None,
        restore_best_weights=True,
)

In [ ]:
for data_name in tqdm(data_names):
    origin_data_name = data_name.replace('adversarial_', '')


    path = f'./data/{origin_data_name}/{origin_data_name}.txt'
    file_list = glob(path)
    data = pd.read_csv(file_list[0], header=None)
    subset = data.iloc[:10000,:20]

    
    for loss_name, loss in tqdm(zip(loss_names, losses)):
        with open(f'./adversarial_data/bim/{origin_data_name}/{data_name}_{loss_name}_epoch_{epoch}_no_sign_{model_name}.pkl', 'rb') as file:
            Xs = pickle.load(file)
            file.close

        with open(f'./adversarial_data/bim/{origin_data_name}/{data_name}_{loss_name}_epoch_{epoch}_no_sign_X_Y_{model_name}.pkl', 'rb') as file:
            X, y, Y = pickle.load(file)
            file.close
    

        for X, e in zip(Xs, epsilons):
            # Fit the model
            model = custom_TCN(
                y=subset.values,
                forecast_period=forecast_period,
                lookback_period=lookback_period,
                kernel_size=2,
                filters=4,
                gru_units=4,
                skip_gru_units=3,
                skip=50,
                lags=forecast_period,
                scaler=scaler,
                loss = loss,
                adversarial_checker = True,
                X=X,
                Y=Y
            )
            history = model.fit(
                loss=loss,
                learning_rate=learning_rate, # origin은 0.01
                batch_size=2048 * 3,
                epochs=epoch,
                verbose=1,
                validation_split=0.3,
                callbacks=[early_stopping],
                metric=metric
            )
            model.load_weights(f'model/{origin_data_name}_{scaler}_epoch_{epoch}_{loss_name}_{model_name}.h5')

            predictions = model.predict_1(X)    

            ep = '{0:0.1f}'.format(e)
            for i in range(y.shape[1]):
                plt.figure(figsize=(20,10))
                plt.title(f'Target {i} epsilon {ep}')
                plt.plot(predictions['actual_' + str(i + 1)], label='actual')
                plt.plot(predictions['predicted_' + str(i + 1)], label=f'epsilon {ep}')
                plt.legend()
                plt.savefig(f'./result/bim/{data_name}/all_predictions_{data_name}_{loss_name}_{scaler}_epoch_{epoch}_target_{i}_{ep}_no_sign_{model_name}.png', format='png')
                plt.cla()
                plt.close('all')
                plt.clf()

            with open(f'./result/bim/{data_name}/all_predictions_{data_name}_{loss_name}_{scaler}_epoch_{epoch}_{ep}_no_sign_{model_name}.pkl', 'wb') as file:
                pickle.dump(predictions, file)
                file.close


            gc.collect()


            del model, predictions
        del Xs, X, y, Y
    del data, subset